In [1]:
import os
os.environ['CUBLAS_WORKSPACE_CONFIG'] = ':4096:8'

In [2]:
import pandas as pd
import numpy as np
import re 
import torch
from transformers import AutoTokenizer, AutoModel
from collections import Counter

import torch
import torch.nn.functional as F

from tqdm import tqdm

/home/g23tka04/miniconda3/envs/graph/lib/python3.10/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
file_path_training = './dataset/train_data_datafinder.jsonl'

file_path_testing = './dataset/test_data_datafinder.jsonl'

df = pd.read_json(file_path_training, lines=True)
df_test = pd.read_json(file_path_testing, lines=True)

df_dataset_information = pd.read_json('./dataset/dataset_search_collection.jsonl', lines=True)

df.drop_duplicates(subset=['paper_id'], inplace=True)

df_test = df_test.map(lambda x: np.nan if x == [] else x)
df_test = df_test.dropna(ignore_index = True)

In [4]:
pattern = (
    r'\\\(.*?\\\)|'  
    r'\r\n|'  
    r'\*\*|'  
    r'\$.*?\$|'  
    r'\\\[.*?\\\]|' 
    r'https?://\S+|' 
    r'www\.\S+|'  
    r'ftp://\S+|'  
    r'\\begin\{equation\}.*?\\end\{equation\}|'  
    r'\\[a-zA-Z]+\*?(?:\[[^\]]*\])?(?:\{[^}]*\})?|' 
    r'\\langle.*?\\rangle|' 
    r'https?://[^\s]+(?:[\s\.,]|$)|' 
    r'www\.[^\s]+(?:[\s\.,]|$)'  
    r'\[Image Source\: \[.*?\]|'
    r'\(Image Source\: \[|'
    r'\[Image Source\: \[|'
    r'\(Source\: \[.*?\]|'
    r'Source\: \[|'  
    r'\(\s*/paper/[^)]+\s*\)'
    
)  

df['abstract'] = df['abstract'].apply(lambda x: re.sub(pattern, '', x))

df_test['abstract'] = df_test['abstract'].apply(lambda x: re.sub(pattern, '', x))


df_graph = df[['title', 'abstract', 'paper_id', 'inbound_citations', 'outbound_citations', 'positives', 'negatives', 'query', 'keyphrase_query']] 

df_test_graph = df_test[['documents', 'abstract', 'query', 'keyphrase_query']]

df_graph['query'] = df_graph.apply(lambda row: row['query'].replace('[DATASET]', str(row['positives'])), axis=1)

/tmp/ipykernel_454206/3707515467.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph['query'] = df_graph.apply(lambda row: row['query'].replace('[DATASET]', str(row['positives'])), axis=1)


In [5]:
pprIdx = {paper_id: idx for idx, paper_id in enumerate(df['paper_id'])}

print(pprIdx)

{191171724: 0, 476005: 1, 195798916: 2, 10136520: 3, 209515237: 4, 49741971: 5, 86816039: 6, 49189997: 7, 207847188: 8, 3964265: 9, 5941931: 10, 196831251: 11, 67856013: 12, 67856708: 13, 211227: 14, 7459313: 15, 3528835: 16, 129945012: 17, 52135124: 18, 208909802: 19, 1981547: 20, 29151476: 21, 29153495: 22, 147703871: 23, 147703900: 24, 195873983: 25, 17062535: 26, 202719239: 27, 10107684: 28, 202661005: 29, 202785103: 30, 202785879: 31, 55702087: 32, 55702202: 33, 14888175: 34, 4162896: 35, 1729289: 36, 16146252: 37, 150373937: 38, 54476205: 39, 14503974: 40, 6913648: 41, 16153365: 42, 12637240: 43, 127980590: 44, 208006462: 45, 52194200: 46, 196622244: 47, 49299822: 48, 6233645: 49, 3462863: 50, 19196695: 51, 211252644: 52, 44167998: 53, 44168818: 54, 214623356: 55, 208921231: 56, 1437449: 57, 3493846: 58, 13106059: 59, 51606370: 60, 195767033: 61, 14004780: 62, 921723: 63, 203706642: 64, 215548043: 65, 16286760: 66, 4379364: 67, 207942291: 68, 104291969: 69, 196212389: 70, 1374515

In [6]:
pprIdx_test = {index: idx for idx, (index, row) in enumerate(df_test.iterrows())}

print(pprIdx_test)

{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 12: 12, 13: 13, 14: 14, 15: 15, 16: 16, 17: 17, 18: 18, 19: 19, 20: 20, 21: 21, 22: 22, 23: 23, 24: 24, 25: 25, 26: 26, 27: 27, 28: 28, 29: 29, 30: 30, 31: 31, 32: 32, 33: 33, 34: 34, 35: 35, 36: 36, 37: 37, 38: 38, 39: 39, 40: 40, 41: 41, 42: 42, 43: 43, 44: 44, 45: 45, 46: 46, 47: 47, 48: 48, 49: 49, 50: 50, 51: 51, 52: 52, 53: 53, 54: 54, 55: 55, 56: 56, 57: 57, 58: 58, 59: 59, 60: 60, 61: 61, 62: 62, 63: 63, 64: 64, 65: 65, 66: 66, 67: 67, 68: 68, 69: 69, 70: 70, 71: 71, 72: 72, 73: 73, 74: 74, 75: 75, 76: 76, 77: 77, 78: 78, 79: 79, 80: 80, 81: 81, 82: 82, 83: 83, 84: 84, 85: 85, 86: 86, 87: 87, 88: 88, 89: 89, 90: 90, 91: 91, 92: 92, 93: 93, 94: 94, 95: 95, 96: 96, 97: 97, 98: 98, 99: 99, 100: 100, 101: 101, 102: 102, 103: 103, 104: 104, 105: 105, 106: 106, 107: 107, 108: 108, 109: 109, 110: 110, 111: 111, 112: 112, 113: 113, 114: 114, 115: 115, 116: 116, 117: 117, 118: 118, 119: 119, 120: 120, 121: 121,

In [7]:
dataset_to_label = {} 
label_counter = 0

for datasets in df_graph['positives']:
    for dataset in datasets:
        if dataset not in dataset_to_label:
            dataset_to_label[dataset] = label_counter
            label_counter += 1

print(dataset_to_label)
print(label_counter)

{'CIFAR-10': 0, 'ImageNet': 1, 'CAT2000': 2, 'Hopkins155': 3, 'VRD': 4, 'Middlebury': 5, 'KITTI': 6, 'LAMA': 7, 'COCO': 8, 'Jester': 9, 'JFLEG': 10, 'IAM': 11, 'Set5': 12, 'AFW': 13, 'FRGC': 14, 'WikiBio': 15, 'Cora': 16, 'BSDS500': 17, 'MovieLens': 18, 'COMA': 19, 'UCF101': 20, 'CARLA': 21, 'Birdsnap': 22, 'PACS': 23, 'VCR': 24, 'ARC': 25, 'MultiRC': 26, 'AudioSet': 27, 'Flickr30k': 28, 'CelebA': 29, 'MuJoCo': 30, 'ReferItGame': 31, 'WebText': 32, 'WikiText-103': 33, 'DeepFashion': 34, 'DRCD': 35, 'NewsQA': 36, 'SQuAD': 37, 'WN18': 38, 'AFLW': 39, 'Helen': 40, 'SumMe': 41, 'Django': 42, 'CompCars': 43, 'ETH': 44, 'Caltech-101': 45, 'LFPW': 46, 'Cityscapes': 47, 'RaFD': 48, 'ECSSD': 49, 'SimpleQuestions': 50, 'QNLI': 51, 'MRPC': 52, 'RACE': 53, 'GLUE': 54, 'SNLI': 55, 'DRIVE': 56, 'ShapeNet': 57, 'MultiNLI': 58, 'DAQUAR': 59, 'IEMOCAP': 60, 'VIPeR': 61, 'STARE': 62, 'ShanghaiTech': 63, 'Pix3D': 64, 'CAD-120': 65, 'HandNet': 66, 'CoQA': 67, 'HMDB51': 68, 'FaceForensics': 69, 'DAVIS': 70

In [8]:
for datasets in df_graph['negatives']:
    for dataset in datasets:
        if dataset not in dataset_to_label:
            dataset_to_label[dataset] = label_counter
            label_counter += 1
        
print(label_counter)

3105


In [9]:
dataset_idcontent_all = []

for _, row in df_dataset_information.iterrows():
    dataset_name = row['id']
    dataset_content = row['contents']
    dataset_title = row['title']
    dataset_structured_data = row['structured_info']
    if dataset_name in dataset_to_label.keys():
        dataset_target_id = dataset_to_label[dataset_name]
        dataset_idcontent_all.append((dataset_target_id, dataset_name, dataset_content, dataset_title, dataset_structured_data))

df_dataset_idcontent = pd.DataFrame(dataset_idcontent_all, columns=['dataset_target_id', 'dataset_name', 'dataset_content', 'dataset_title', 'dataset_structured_data' ])

df_dataset_idcontent.sort_values(by='dataset_target_id', inplace=True, ignore_index=True)
df_dataset_idcontent.drop_duplicates(subset=['dataset_target_id'], inplace=True, ignore_index=True)

In [10]:

pattern_dataset = (
    r'\\\(.*?\\\)|'
    r'\r\n|'  
    r'\*\*|' 
    r'\*|'  
    r'\$.*?\$|'  
    r'\\\[.*?\\\]|' 
    r'\\[a-zA-Z]+\*?(?:\[[^\]]*\])?(?:\{[^}]*\})?|' 
    r'\n|'
    r'\n+|'
    r'\(https?://\S+\)|' 
    r'(Source:|Image Source:|Image:|NOTE: ).*|'
    r'\[\'|'
    r'\'\]|'
    r'\{|'
    r'\}|'
    
)  

def clean_text(text):
    # Remove unwanted patterns
    cleaned = re.sub(pattern_dataset, '', text)
    # Condense multiple whitespace characters into a single space
    cleaned = re.sub(r'\s+', ' ', cleaned)
    # Remove leading and trailing whitespace
    cleaned = cleaned.strip()
    return cleaned

df_dataset_idcontent['dataset_content'] = df_dataset_idcontent['dataset_content'].apply(clean_text)
df_dataset_idcontent['dataset_structured_data'] = df_dataset_idcontent['dataset_structured_data'].apply(clean_text)
df_graph['query'] = df_graph['query'].apply(clean_text)

/tmp/ipykernel_454206/3924259041.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph['query'] = df_graph['query'].apply(clean_text)


In [11]:
class Textual_Feature(torch.nn.Module):
    def __init__(self, checkpoint= 'allenai/scibert_scivocab_uncased', device='cuda'):
        super().__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.lm = AutoModel.from_pretrained(checkpoint)
        self.device = device
        self.lm.to(self.device)

    def encode(self, paper_text):
        tok_text = self.tokenizer(paper_text,
                             truncation = True,
                             max_length = 512, 
                             padding = 'max_length',
                             return_tensors='pt').to(self.device)
        
            
        tok_text = {k: v.to(self.device) for k, v in tok_text.items()}
    
        with torch.no_grad():
            outputs = self.lm(**tok_text)
        embeddings = torch.mean(outputs.last_hidden_state, dim=1)
        return embeddings

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

tqdm.pandas()
textual_feature = Textual_Feature()

paper_texts = df_graph['keyphrase_query'].str.lower() + '[SEP]' + df_graph['query'].str.lower() + '[SEP]' +  df_graph['title'].str.lower() + '[SEP]' + df_graph['abstract'].str.lower()
test_text = df_test['query'].str.lower() + '[SEP]' + df_test['keyphrase_query'].str.lower() + '[SEP]' + df_test['abstract'].str.lower()

dataset_text =  df_dataset_idcontent['dataset_title'].str.lower() + '[SEP]' + df_dataset_idcontent['dataset_name'].str.lower() + '[SEP]' + df_dataset_idcontent['dataset_content'].str.lower() + '[SEP]' + df_dataset_idcontent['dataset_structured_data'].str.lower() 

token_paper = paper_texts.progress_apply(lambda text: textual_feature.encode(text))
token_dataset = dataset_text.progress_apply(lambda text: textual_feature.encode(text))
token_test = test_text.progress_apply(lambda text: textual_feature.encode(text))

/home/g23tka04/miniconda3/envs/graph/lib/python3.10/site-packages/huggingface_hub/file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/g23tka04/miniconda3/envs/graph/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/g23tka04/miniconda3/envs/graph/lib/python3.10/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(
/home/g23tka04/miniconda3/envs/graph/lib/python3.10/site-packages/transformers/modeling_utils.py:484: FutureWarning: You are using `torch

In [13]:
dataset_embeddings_tensor = torch.vstack(list(token_dataset))
paper_embeddings_tensor = torch.vstack(list(token_paper))
test_embeddings_tensor = torch.vstack(list(token_test))

In [14]:
all_paper_embeddings_tensor = torch.cat((paper_embeddings_tensor, test_embeddings_tensor), dim=0)

In [15]:
node_label_positive_all = []
node_label_negative_all = []
node_label_test = []



for datasets in df_graph['positives']:
    numerical_labels = [dataset_to_label[dataset] for dataset in datasets]
    node_label_positive_all.append(numerical_labels)
    
    
for datasets in df_graph['negatives']:
    numerical_labels = [dataset_to_label[dataset] for dataset in datasets]
    node_label_negative_all.append(numerical_labels)
    
for datasets in df_test_graph['documents']:
    numerical_labels = [dataset_to_label[dataset] for dataset in datasets]
    node_label_test.append(numerical_labels)


df_graph['positive label'] = node_label_positive_all
df_graph['negative label'] = node_label_negative_all
df_test_graph['dataset label'] = node_label_test

/tmp/ipykernel_454206/2345754625.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph['positive label'] = node_label_positive_all
/tmp/ipykernel_454206/2345754625.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_graph['negative label'] = node_label_negative_all
/tmp/ipykernel_454206/2345754625.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https:/

In [16]:
edges_p2d = []
edges_p2d_positive= []
edges_p2d_negative= []

outbound_citation = []

train_dataset_usage_counter = Counter()

all_triplets = []

for _, row in df_graph.iterrows():
     source_node_index = pprIdx[row['paper_id']]

     pos_idx = []
     neg_idx = []
     
     positive =  row['positive label']

     negative = row['negative label']

     for dataset_id in positive:
          edges_p2d.append([source_node_index, dataset_id])
          edges_p2d_positive.append([source_node_index, dataset_id])
          train_dataset_usage_counter[dataset_id] += 1
          pos_idx.append(dataset_id)

     for dataset_id in negative:
          edges_p2d.append([source_node_index, dataset_id])
          edges_p2d_negative.append([source_node_index, dataset_id])
          neg_idx.append(dataset_id)
          
     for pos in pos_idx:
          for neg in neg_idx:
               all_triplets.append([source_node_index, pos, neg])

In [17]:
edges_p2d_test = []
unique_values = set()
count_dataset = []
test_dataset_usage_counter = Counter()

for _, row in df_test_graph.iterrows():
    source_node_index = pprIdx_test[_]+ 17397
    dataset =  row['dataset label']
    
    for dataset_id in dataset:
        if dataset_id not in count_dataset:
            count_dataset.append(dataset_id)
        edges_p2d_test.append([source_node_index, dataset_id])
        test_dataset_usage_counter[dataset_id] += 1    

In [18]:
edge_index_p2d_positive = torch.tensor(edges_p2d_positive, dtype=torch.long).t().contiguous()
edge_index_p2d_negative = torch.tensor(edges_p2d_negative, dtype=torch.long).t().contiguous()
positive_negative_index = torch.cat((edge_index_p2d_positive, edge_index_p2d_negative), dim=1)

edge_index_test = torch.tensor(edges_p2d_test, dtype=torch.long).t().contiguous()

In [19]:
torch.cuda.empty_cache()
from tqdm import tqdm
import random
from torch.optim.lr_scheduler import SequentialLR, LinearLR, CosineAnnealingLR

import torch_geometric.transforms as T
from torch_geometric.loader import NeighborLoader, DataLoader

from torch_geometric.nn import SAGEConv, to_hetero
from torch_geometric.data import HeteroData
from evaluation import ranking_evaluation

In [20]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  

    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

seed_num = 1

set_seed(seed_num)
torch.use_deterministic_algorithms(True)
g = torch.Generator()
g.manual_seed(seed_num)

In [21]:
train_data = HeteroData()
train_data['paper'].x = paper_embeddings_tensor
train_data['dataset'].x = dataset_embeddings_tensor
train_data['paper', 'use', 'dataset'].edge_index = positive_negative_index
train_data['paper', 'use', 'dataset'].edge_label_index = train_data['paper', 'use', 'dataset'].edge_index


train_data = T.ToUndirected()(train_data)

In [22]:
test_data = HeteroData()
test_data['paper'].x = all_paper_embeddings_tensor
test_data['dataset'].x = dataset_embeddings_tensor
test_data['paper', 'use', 'dataset'].edge_index = positive_negative_index
test_data['paper', 'use', 'dataset'].edge_label_index = test_data['paper', 'use', 'dataset'].edge_index

test_data = T.ToUndirected()(test_data)

In [23]:
train_data.to(device)
test_data.to(device)

print(train_data)
print(test_data)

HeteroData(
  paper={ x=[17397, 768] },
  dataset={ x=[3105, 768] },
  (paper, use, dataset)={
    edge_index=[2, 142562],
    edge_label_index=[2, 142562],
  },
  (dataset, rev_use, paper)={ edge_index=[2, 142562] }
)
HeteroData(
  paper={ x=[17781, 768] },
  dataset={ x=[3105, 768] },
  (paper, use, dataset)={
    edge_index=[2, 142562],
    edge_label_index=[2, 142562],
  },
  (dataset, rev_use, paper)={ edge_index=[2, 142562] }
)


In [24]:
class TripletDataset(torch.utils.data.Dataset):
    def __init__(self, triplets):
        self.triplets = triplets

    def __len__(self):
        return len(self.triplets)

    def __getitem__(self, idx):
        return self.triplets[idx]

In [25]:
triplet_loader = DataLoader(
    TripletDataset(all_triplets),
    batch_size=8192,
    shuffle=True,
    generator=g,
)

In [26]:
class GNN(torch.nn.Module):
    def __init__(self, hidden_channels, input_dim = 768):
        super().__init__()
        self.conv1 = SAGEConv((-1, -1), hidden_channels)
        self.conv2 = SAGEConv((-1, -1), hidden_channels)
        self.norm1 = torch.nn.LayerNorm(hidden_channels)
        self.norm2 = torch.nn.LayerNorm(hidden_channels)
        
        self.res_proj1 = (torch.nn.Linear(input_dim, hidden_channels) 
                          if input_dim != hidden_channels else torch.nn.Identity())
        self.res_proj2 = torch.nn.Identity()
        
    def forward(self, x, edge_index):
        
        identity = self.res_proj1(x)

        out = self.conv1(x, edge_index)
        out = self.norm1(out.relu())
        out = out + identity  

        identity2 = self.res_proj2(out)
        
        # Second convolution
        out = self.conv2(out, edge_index)
        out = self.norm2(out.relu())
        out = out + identity2 

        return out
    
class TripletDecoder(torch.nn.Module):
    def __init__(self, hidden_channels, input_dim = 1152):
        super().__init__()
        self.paper_proj = torch.nn.Linear(input_dim, hidden_channels)
        self.dataset_proj = torch.nn.Linear(input_dim, hidden_channels)
        
    def forward(self, rec_x_dict, cl_x_dict, x_dict, cl=False):
        paper_emb = self.paper_proj(torch.cat((rec_x_dict['paper'], x_dict['paper']), dim=1))
        dataset_emb = self.dataset_proj(torch.cat((rec_x_dict['dataset'], x_dict['dataset']), dim=1))
        if cl:
            cl_paper_emb = self.paper_proj(torch.cat((cl_x_dict['paper'], x_dict['paper']), dim=1))
            cl_dataset_emb = self.dataset_proj(torch.cat((cl_x_dict['dataset'], x_dict['dataset']), dim=1))
            return paper_emb, dataset_emb, cl_paper_emb, cl_dataset_emb
        return paper_emb, dataset_emb


class Model(torch.nn.Module):
    def __init__(self, hidden_channels, data):
        super().__init__()
        self.data = data
        self.encoder = GNN(hidden_channels)
        # self.linear = torch.nn.Linear(hidden_channels, emb_size)
        self.encoder = to_hetero(self.encoder, self.data.metadata(), aggr='sum')
        self.decoder = TripletDecoder(hidden_channels)

    def forward(self, x_dict, edge_index_dict, cl=False): 
        
        cl_edge_index_dict = self.apply_cdmin_edge_dropout(x_dict, edge_index_dict, drop_ratio=0.2)
        
        cl_x_dict = self.encoder(x_dict, cl_edge_index_dict)
        cl_x_dict = {k: F.normalize(v, p=2, dim=-1) for k, v in cl_x_dict.items()}

        rec_x_dict = self.encoder(x_dict, edge_index_dict)
        rec_x_dict = {k: F.normalize(v, p=2, dim=-1) for k, v in rec_x_dict.items()}
        
        return self.decoder(rec_x_dict, cl_x_dict, x_dict, cl)
    
    def apply_cdmin_edge_dropout(self, x_dict, edge_index_dict, drop_ratio):
        edge_index = edge_index_dict['paper', 'use', 'dataset']
        paper_ids = edge_index[0]
        dataset_ids = edge_index[1]

        cosine_sims = (x_dict["paper"][paper_ids] * x_dict["dataset"][dataset_ids]).sum(dim=-1)  # (num_edges,)
        
        num_edges = cosine_sims.size(0)
        num_drop = int(drop_ratio * num_edges)

        # Drop lowest similarity edges
        _, drop_idx = torch.topk(cosine_sims, k=num_drop, largest=False)
        keep_mask = torch.ones(num_edges, dtype=torch.bool, device=edge_index.device)
        keep_mask[drop_idx] = False

        # Filter edge index
        new_edge_index = edge_index[:, keep_mask]
        
        # Update edge_index_dict with new sparse graph
        new_edge_index_dict = edge_index_dict.copy()
        new_edge_index_dict['paper', 'use', 'dataset'] = new_edge_index
        
        return new_edge_index_dict

model = Model(hidden_channels=384, data=train_data).to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-3)

In [27]:

# Define the number of warmup and total epochs
warmup_epochs = 5
total_epochs = 40

# Create the linear warmup scheduler
linear_scheduler = LinearLR(optimizer, start_factor=0.1, total_iters=warmup_epochs)

# Create the cosine annealing scheduler
cosine_scheduler = CosineAnnealingLR(optimizer, T_max=total_epochs - warmup_epochs)

# Combine the schedulers
scheduler = SequentialLR(optimizer, schedulers=[linear_scheduler, cosine_scheduler], milestones=[warmup_epochs])

In [28]:
def get_batch_nodes(triplet_batch):
    
    paper_ids = triplet_batch[0]
    pos_ids   = triplet_batch[1]
    neg_ids   = triplet_batch[2]

    all_papers = list(set(paper_ids.tolist()))
    all_datasets = list(set(torch.cat((pos_ids, neg_ids), dim=0).tolist()))

    return all_papers, all_datasets

def get_subgraph(paper_ids, full_graph):
    
    input_paper_nodes = torch.tensor(paper_ids)
    if input_paper_nodes.dim() == 0:
        input_paper_nodes = input_paper_nodes.unsqueeze(0)

    loader = NeighborLoader(
        data=full_graph,
        input_nodes=['paper', input_paper_nodes],
        num_neighbors=[20, 15], 
        shuffle=False,
        batch_size=len(paper_ids)
    )
    
    return next(iter(loader))  
    
def map_global_to_local(triplets, batch):
    paper_nid = batch['paper'].n_id.tolist()
    dataset_nid = batch['dataset'].n_id.tolist()

    g2l_paper = {g: i for i, g in enumerate(paper_nid)}
    g2l_dataset = {g: i for i, g in enumerate(dataset_nid)}
    
    paper_idx = []
    pos_idx = []
    neg_idx = []
    for anchor_id, pos_id, neg_id in zip(*triplets):
        paper_idx.append(g2l_paper[int(anchor_id)])
        pos_idx.append(g2l_dataset[int(pos_id)])
        neg_idx.append(g2l_dataset[int(neg_id)])
        
    return paper_idx, pos_idx, neg_idx

In [29]:
structure_temp = 0.005
text_temp = 0.08
    

def l2_reg_loss(reg, *args):
    emb_loss = 0
    for emb in args:
        emb_loss += torch.norm(emb, p=2)/emb.shape[0]
    return emb_loss * reg

def InfoNCE(view1, view2, temperature: float, b_cos: bool = True):
    if b_cos:
        view1, view2 = F.normalize(view1, dim=1), F.normalize(view2, dim=1)

    pos_score = (view1 @ view2.T) / temperature
    score = torch.diag(F.log_softmax(pos_score, dim=1))
    return -score.mean()

def cal_cl_loss(idx, paper_view1, paper_view2, dataset_view1, dataset_view2):
    p_idx = torch.unique(torch.Tensor(idx[0]).type(torch.long)).cuda()
    d_idx = torch.unique(torch.Tensor(idx[1]).type(torch.long)).cuda()
    
    paper_cl_loss = InfoNCE(paper_view1[p_idx], paper_view2[p_idx], structure_temp)
    dataset_cl_loss = InfoNCE(dataset_view1[d_idx], dataset_view2[d_idx], structure_temp)
    
    return paper_cl_loss + dataset_cl_loss

def text_cl(paper, dataset):
    
    loss = InfoNCE(paper, dataset, text_temp)
    
    return loss

def margin_ranking(paper_emb, pos_emb, neg_emb):
    pos_score = torch.sum(paper_emb * pos_emb, dim=1)
    neg_score = torch.sum(paper_emb * neg_emb, dim=1)
    

    target = torch.ones_like(pos_score)

    ranking_loss_fn = torch.nn.MarginRankingLoss(margin=1)
    ranking_loss = ranking_loss_fn(pos_score, neg_score, target)
    
    return ranking_loss

In [30]:

base_cl_rate = 2.0
base_text_cl_rate = 0.8
margin_loss_rate = 2.0

warmup = 5

reg = 0.00001

def train(epoch):
    model.train()
    total_loss = 0.0
    total_examples = 0
    uncounted = 0
    
    
    cl_rate = min(epoch / warmup, 1.0) * base_cl_rate
    text_cl_rate = min(epoch / warmup, 1.0) * base_text_cl_rate
    
    
    for n, batch_triplets in (pbar := tqdm(enumerate(triplet_loader))):
    
        paper_ids, dataset_ids = get_batch_nodes(batch_triplets)
        batch = get_subgraph(paper_ids, train_data)
        
        
        try:
            paper_idx, pos_idx, neg_idx = map_global_to_local(batch_triplets, batch)
        except KeyError as e:
            # print(f"[Warning] Skipping batch due to missing node: {e}")
            uncounted += 1
            continue

        
        rec_paper_emb, rec_dataset_emb, cl_paper_emb, cl_dataset_emb = model(
            batch.x_dict, 
            batch.edge_index_dict, 
            cl=True,
        )

        paper_emb, pos_emb, neg_emb = rec_paper_emb[paper_idx], rec_dataset_emb[pos_idx], rec_dataset_emb[neg_idx]
        
        text_cl_loss = text_cl_rate * text_cl(paper_emb, pos_emb)

        cl_loss = cl_rate * cal_cl_loss([paper_idx, pos_idx], rec_paper_emb, cl_paper_emb, rec_dataset_emb, cl_dataset_emb)
        
        margin_ranking_loss = margin_loss_rate * margin_ranking(paper_emb, pos_emb, neg_emb)

        

        batch_loss = l2_reg_loss(reg, paper_emb, pos_emb) + cl_loss + text_cl_loss + margin_ranking_loss
        
        optimizer.zero_grad()
        batch_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=5.0)
        optimizer.step()
        scheduler.step()

        total_loss += batch_loss.item()
        total_examples += 1
        
    return total_loss / total_examples, uncounted

In [31]:
import heapq

def find_k_largest(K, candidates):

    n_candidates = []
    for did, score in enumerate(candidates[:K]):
        n_candidates.append((score.item(), did))
    heapq.heapify(n_candidates)
    for did, score in enumerate(candidates[K:]):
        score_val = score.item()
        if score > n_candidates[0][0]:
            heapq.heapreplace(n_candidates, (score, did + K))
    n_candidates.sort(key=lambda d: d[0], reverse=True)
    ids = [item[1] for item in n_candidates]
    k_largest_scores = [item[0] for item in n_candidates]
    return ids, k_largest_scores

In [32]:
import faiss

def test_full_batch(epoch, ground_truth, k=5):
    results = []
    all_scores = []

    model.eval()
    with torch.no_grad():
        ori_paper_emb, dataset_emb = model(
            test_data.x_dict, 
            test_data.edge_index_dict,
        )
        
        test_papers_idx = edge_index_test[0]

        paper_nids, inv = torch.unique(test_papers_idx, sorted=True, return_inverse=True)

        paper_emb = ori_paper_emb[paper_nids] 

        paper_var = paper_emb.var(dim=0).mean()
        dataset_var = dataset_emb.var(dim=0).mean()
         
        print("paper_emb variance across papers:", paper_emb.var(dim=0).mean())
        print("dataset_emb variance:", dataset_emb.var(dim=0).mean())

        paper_np = paper_emb.cpu().numpy().astype('float32')
        dataset_np = dataset_emb.cpu().numpy().astype('float32')

        dim = dataset_np.shape[1]
        index = faiss.IndexFlatIP(dim)  
        index.add(dataset_np)

        scores, indices = index.search(paper_np, k) 
        scores = torch.tensor(scores)
        results = torch.tensor(indices)
        print(results)
        
        flat_results = results.view(-1)

        unique_datasets = torch.unique(flat_results)

        print(f"Total unique datasets recommended in Top-5 across all papers: {len(unique_datasets)}")
        print(f"Sample of unique dataset IDs: {unique_datasets.tolist()}")

        all_scores = torch.tensor(index.search(paper_np, dataset_np.shape[0])[0])

        variance_score = all_scores.var(dim=0).mean()
        print(f'Variance_score: {torch.round(variance_score, decimals=5)}')

        origin_dict = {i: {item: 1 for item in items} for i, items in enumerate(ground_truth)}

        res_dict = {i: [(int(item), 0.0) for item in row.tolist()] for i, row in enumerate(results)}

        measure = ranking_evaluation(origin_dict, res_dict, [5])

        performance = {k: float(v) for m in measure[1:] for k, v in [m.strip().split(':')]}

        return performance


In [33]:
from collections import defaultdict

# edge_label_index[0] -> users, edge_label_index[1] -> items
paper_to_dataset = defaultdict(set)
for p, i in zip(edge_index_test[0].tolist(), edge_index_test[1].tolist()):
    paper_to_dataset[p].add(int(i))
    
ground_truth = []

for paper, dataset in paper_to_dataset.items():
    ground_truth.append(dataset)
    
ground_truth = [list(items) for items in ground_truth]

In [34]:
early_stopping = 0

best_map = 0
best_precision = 0
best_recall = 0
best_ndcg = 0
best_pred_index_mat = []
best_epoch = 0

bestPerformance = []

for epoch in range(1, 40):
    train_loss, uncounted= train(epoch)
    print(f'Epoch: {epoch:02d}, Loss: {train_loss:.4f}, Uncounted: {uncounted}')
    result = test_full_batch(
        epoch,
        ground_truth,
    )
    
    if bestPerformance:
        count = sum(1 if bestPerformance[1][k] > result[k] else -1 for k in result)
        if count < 0:
            early_stopping = 0
            bestPerformance = [epoch, result]
            torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'performance': result,
            }, 'best_model.pt')
        
            print(f"Saved best model at epoch {epoch}")
            
        else:
            early_stopping += 1

    else:
        bestPerformance = [epoch, result]
        early_stopping = 0
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict(),
            'performance': result,
            }, 'best_model.pt')
        
        print(f"Saved initial run model at epoch {epoch}")
        
    print('-' * 80)
    print(f'Real-Time Ranking Performance (Top-5 Dataset Recommendation)')
    measure_str = ', '.join([f'{k}: {v}' for k, v in result.items()])
    print(f'*Current Performance*\nEpoch: {epoch}, {measure_str}')
    bp = ', '.join([f'{k}: {v}' for k, v in bestPerformance[1].items()])
    print(f'*Best Performance*\nEpoch: {bestPerformance[0]}, {bp}')
    print('-' * 80)
    
    if early_stopping >= 7:
            print('EARLY STOPPING TRIGGERED!!!!!')
            break

18it [00:05,  3.24it/s]


Epoch: 01, Loss: 3.0495, Uncounted: 0
paper_emb variance across papers: tensor(0.0641, device='cuda:0')
dataset_emb variance: tensor(0.0416, device='cuda:0')
tensor([[   8,    6,    1,  701,   17],
        [   1,    8,    6,   48, 2142],
        [   8,    1,   37,   18, 2142],
        ...,
        [   1,    0, 2142,    8, 1879],
        [1770,    1,   37,   93, 2000],
        [   1,    8, 1770,   37, 2142]])
Total unique datasets recommended in Top-5 across all papers: 37
Sample of unique dataset IDs: [0, 1, 6, 8, 13, 17, 18, 29, 30, 37, 48, 55, 56, 63, 74, 76, 79, 93, 101, 108, 127, 135, 172, 284, 367, 401, 419, 693, 701, 1198, 1484, 1770, 1879, 1897, 2000, 2142, 2211]
Variance_score: 19.04360008239746
Saved initial run model at epoch 1
--------------------------------------------------------------------------------
Real-Time Ranking Performance (Top-5 Dataset Recommendation)
*Current Performance*
Epoch: 1, Hit Ratio: 0.186, Precision: 0.092, Recall: 0.179, MAP: 0.115, MRR: 0.201, NDC

18it [00:03,  4.98it/s]


Epoch: 02, Loss: 2.7499, Uncounted: 0
paper_emb variance across papers: tensor(0.0705, device='cuda:0')
dataset_emb variance: tensor(0.0400, device='cuda:0')
tensor([[   8,    6,    1,   47, 3033],
        [   1,    8,    6,   17,  149],
        [  37,  177,  135, 1837,   75],
        ...,
        [   0,   29,    1, 1879, 2142],
        [  37,  177,  610,   58, 1270],
        [  37,  177,   58,   75,   55]])
Total unique datasets recommended in Top-5 across all papers: 125
Sample of unique dataset IDs: [0, 1, 2, 5, 6, 8, 10, 12, 13, 16, 17, 18, 20, 25, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39, 40, 45, 47, 48, 55, 57, 58, 60, 63, 65, 68, 70, 72, 73, 74, 75, 78, 79, 83, 84, 85, 90, 93, 97, 101, 106, 108, 109, 111, 112, 119, 122, 127, 128, 135, 142, 146, 148, 149, 159, 164, 172, 174, 177, 194, 207, 210, 221, 260, 262, 265, 283, 284, 302, 303, 335, 345, 401, 437, 455, 533, 569, 610, 627, 701, 719, 763, 766, 820, 901, 1063, 1099, 1227, 1270, 1302, 1385, 1392, 1484, 1770, 1777, 1793, 1830, 183

18it [00:03,  5.39it/s]


...

Epoch: 16, Loss: 5.2202, Uncounted: 0
paper_emb variance across papers: tensor(0.0648, device='cuda:0')
dataset_emb variance: tensor(0.0475, device='cuda:0')
tensor([[  47,  186, 2007, 1063, 2915],
        [2804,  134, 2841, 2942, 2114],
        [  37,   75,  283,   36, 1933],
        ...,
        [  29,    0,  146, 1879, 1392],
        [  55,   58,   37,  284,   54],
        [  55,   58,   28,  183,  284]])
Total unique datasets recommended in Top-5 across all papers: 361
Sample of unique dataset IDs: [0, 1, 2, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 24, 27, 28, 29, 30, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 43, 44, 45, 46, 47, 48, 49, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 68, 70, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 93, 94, 96, 97, 99, 100, 101, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 117, 118, 119, 120, 121, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 137, 138, 141, 142, 144, 146, 149, 154, 155, 159, 160,

18it [00:03,  5.40it/s]


Epoch: 17, Loss: 5.3196, Uncounted: 0
paper_emb variance across papers: tensor(0.0823, device='cuda:0')
dataset_emb variance: tensor(0.0591, device='cuda:0')
tensor([[  47, 2007,  186, 2915,   82],
        [2804, 2841,  134, 1297,   56],
        [  37,   98,   36,   75, 1939],
        ...,
        [   0,   29, 1879,    1,  146],
        [  58,   55,   37,   32,   84],
        [  58,   55,  183,   84,   28]])
Total unique datasets recommended in Top-5 across all papers: 282
Sample of unique dataset IDs: [0, 1, 5, 6, 8, 9, 10, 12, 13, 14, 16, 17, 18, 20, 23, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39, 43, 44, 45, 46, 47, 48, 49, 51, 52, 54, 55, 56, 57, 58, 60, 61, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 93, 96, 98, 99, 101, 104, 106, 107, 108, 109, 111, 112, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 127, 128, 129, 130, 133, 134, 135, 137, 138, 139, 141, 142, 144, 146, 149, 159, 165, 172, 177, 179, 183, 185, 186, 188, 189, 191, 193, 194, 198, 203, 204, 

18it [00:03,  5.07it/s]


Epoch: 18, Loss: 4.9638, Uncounted: 0
paper_emb variance across papers: tensor(0.0806, device='cuda:0')
dataset_emb variance: tensor(0.0576, device='cuda:0')
tensor([[  47, 2007,  186, 2915,   82],
        [2804,  134, 2841, 1297,   56],
        [  37,   98,   75,   36,   67],
        ...,
        [   0,   29, 1879,   30,  146],
        [  58,   55,   84,   37,   32],
        [ 183,   58,   55,   84,   28]])
Total unique datasets recommended in Top-5 across all papers: 326
Sample of unique dataset IDs: [0, 1, 2, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20, 21, 23, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47, 48, 49, 51, 52, 54, 55, 56, 57, 58, 59, 60, 61, 67, 68, 70, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 86, 87, 93, 97, 98, 99, 100, 101, 104, 105, 106, 108, 109, 112, 113, 115, 116, 117, 118, 119, 120, 121, 122, 124, 125, 126, 127, 128, 129, 130, 133, 134, 135, 137, 138, 139, 141, 142, 144, 146, 149, 154, 155, 159, 164, 165, 172, 173, 177, 